# Copy Jars to hdfs

In [ ]:
# Run Thes command on master node of EMR
hdfs dfs -mkdir -p /usr/lib/hudi/
hdfs dfs -mkdir -p /usr/lib/spark/external/lib/
hdfs dfs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar /usr/lib/hudi/hudi-spark-bundle.jar
hdfs dfs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar /usr/lib/spark/external/lib/spark-avro.jar
hdfs dfs -ls /usr/lib/hudi/hudi-spark-bundle.jar /usr/lib/spark/external/lib/spark-avro.jar

In [1]:
%%configure -f
{"conf":{"spark.serializer":"org.apache.spark.serializer.KryoSerializer",
         "spark.sql.hive.convertMetastoreParquet":"false",
         "spk.dynamicAllocation.maxExecutors":"10",
         "spark.jars":"hdfs:///usr/lib/hudi/hudi-spark-bundle.jar,hdfs:///usr/lib/spark/external/lib/spark-avro.jar"
        }
}

In [2]:
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.config.HoodieWriteConfig._
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.DataSourceReadOptions
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.QuickstartUtils._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SaveMode}
import org.apache.spark.sql.catalyst.encoders.RowEncoder
import org.apache.spark.sql.types.{LongType, StringType, StructField, StructType}
import org.apache.spark.sql.SparkSession
import org.apache.log4j.Logger
import org.apache.spark.storage.StorageLevel
import java.util.Date
import java.text.SimpleDateFormat
import org.apache.spark.sql.streaming._
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.streaming.StreamingQueryListener._
import java.util.concurrent.TimeUnit
import org.apache.hudi.keygen._
import java.util._
import java.time._
import java.time.format._

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
16,application_1639488028357_0016,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.config.HoodieWriteConfig._
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.DataSourceReadOptions
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.QuickstartUtils._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SaveMode}
import org.apache.spark.sql.catalyst.encoders.RowEncoder
import org.apache.spark.sql.types.{LongType, StringType, StructField, StructType}
import org.apache.spark.sql.SparkSession
import org.apache.log4j.Logger
import org.apache.spark.storage.StorageLevel
import java.util.Date
import java.text.SimpleDateFormat
import org.apache.spark.sql.streaming._
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.streaming.StreamingQueryListener._
import java.

In [4]:
import spark.implicits._
// Spark Shell -- hardcode these parameters
var s3_bucket = "akshaya-hudi-experiments"
var sourceHudiTableName =  "equity_trade_records_cow"
var targetTableType = "COW"
var targetTableNamePrefix = "hudi_trade_info_derived"
var targetTableName = targetTableNamePrefix + "_cow"
var hudiDatabaseName = "demohudi"
var dsWriteOptionType = DataSourceWriteOptions.COW_STORAGE_TYPE_OPT_VAL
if (targetTableType.equals("COW")) {
      targetTableName = targetTableNamePrefix + "_cow"
      dsWriteOptionType = DataSourceWriteOptions.COW_STORAGE_TYPE_OPT_VAL
} else if (targetTableType.equals("MOR")) {
      targetTableName = targetTableNamePrefix + "_mor"
      dsWriteOptionType = DataSourceWriteOptions.MOR_STORAGE_TYPE_OPT_VAL
}
val sourceHudiTablePath = s"s3://$s3_bucket/demo/hudi/" + sourceHudiTableName

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import spark.implicits._
s3_bucket: String = akshaya-hudi-experiments
sourceHudiTableName: String = equity_trade_records_cow
targetTableType: String = COW
targetTableNamePrefix: String = hudi_trade_info_derived
targetTableName: String = hudi_trade_info_derived_cow
hudiDatabaseName: String = demohudi
dsWriteOptionType: String = COPY_ON_WRITE
sourceHudiTablePath: String = s3://akshaya-hudi-experiments/demo/hudi/equity_trade_records_cow


In [35]:
LocalDateTime.now(ZoneOffset.UTC)
val date=LocalDateTime.now(ZoneOffset.UTC).minusMinutes(20).format(DateTimeFormatter.ofPattern("yyyyMMddHHmmss")) 
spark.read.format("hudi").load(sourceHudiTablePath).createOrReplaceTempView(sourceHudiTableName)
val sql=(s"select max(_hoodie_commit_time) as commitTime from  $sourceHudiTableName where  _hoodie_commit_time < $date ")
val commits = spark.sql(sql).map(k => k.getString(0)).take(10)  
var beginTime =  Optional.ofNullable(commits(0) ).orElse(date)




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res196: java.time.LocalDateTime = 2021-12-14T16:34:49.373
date: String = 20211214161449
sql: String = "select max(_hoodie_commit_time) as commitTime from  equity_trade_records_cow where  _hoodie_commit_time < 20211214161449 "
commits: Array[String] = Array(20211214154500)
beginTime: String = 20211214154500


In [42]:
val hudiTableRecordKey = "record_key"
val hudiTablePrecombineKey = "trade_datetime"
val hudiHiveTablePartitionKey = "symbol,day,hour"
val targetHudiTablePath = s"s3://$s3_bucket/demo/hudi/" + targetTableName
println("hudiDatabaseName.targetTableName:" + hudiDatabaseName+"."+targetTableName)
println("targetHudiTablePath:" + targetHudiTablePath)
println("hudiTablePrecombineKey:" + hudiTablePrecombineKey)    
println("hudiHiveTablePartitionKey:" + hudiHiveTablePartitionKey)

val incrementalDF = (spark.read.format("hudi")
        .option(QUERY_TYPE_OPT_KEY, QUERY_TYPE_INCREMENTAL_OPT_VAL)
        .option(BEGIN_INSTANTTIME_OPT_KEY, beginTime)
        .load(sourceHudiTablePath))
incrementalDF.show()
incrementalDF.printSchema()

val commitTimeDF=incrementalDF.select(unix_timestamp(col("_hoodie_commit_time"),"yyyyMMddHHmmss").as("committime"))
val newbeginTime = commitTimeDF.agg(max(commitTimeDF.col("committime"))).select(from_unixtime(col("max(committime)"),"yyyyMMddHHmmss")).map(k => k.getString(0)).take(1)
val maxTimeStamp=newbeginTime(0)
if(maxTimeStamp !=null){
    beginTime=maxTimeStamp
}
println(beginTime)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hudiTableRecordKey: String = record_key
hudiTablePrecombineKey: String = trade_datetime
hudiHiveTablePartitionKey: String = symbol,day,hour
targetHudiTablePath: String = s3://akshaya-hudi-experiments/demo/hudi/hudi_trade_info_derived_cow
hudiDatabaseName.targetTableName:demohudi.hudi_trade_info_derived_cow
targetHudiTablePath:s3://akshaya-hudi-experiments/demo/hudi/hudi_trade_info_derived_cow
hudiTablePrecombineKey:trade_datetime
hudiHiveTablePartitionKey:symbol,day,hour
incrementalDF: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 15 more fields]
+-------------------+--------------------+--------------------+----------------------+--------------------+---------------+------+--------+-----+----------+--------------------+------------+----------+--------------------+-------------------+---+----+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|        tradeid|symbol|quantity|price|

In [43]:

var result = (incrementalDF.write.format("hudi")
      .options(getQuickstartWriteConfigs)
      .option(TABLE_TYPE_OPT_KEY, dsWriteOptionType)
      .option(TABLE_NAME, targetTableName)
      .option(RECORDKEY_FIELD_OPT_KEY, hudiTableRecordKey)
      .option(PARTITIONPATH_FIELD_OPT_KEY,hudiHiveTablePartitionKey)
      .option(PRECOMBINE_FIELD_OPT_KEY, hudiTablePrecombineKey)
      .option(KEYGENERATOR_CLASS_OPT_KEY, classOf[ComplexKeyGenerator].getName)     
      .option(HIVE_STYLE_PARTITIONING_OPT_KEY, "true")
      .option(HIVE_SYNC_ENABLED_OPT_KEY, "true")
      .option(HIVE_TABLE_OPT_KEY, targetTableName)
      .option(HIVE_PARTITION_FIELDS_OPT_KEY, hudiHiveTablePartitionKey)
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY, classOf[MultiPartKeysValueExtractor].getName)
      .option(HIVE_DATABASE_OPT_KEY, hudiDatabaseName)
      .option("hoodie.metadata.enable", "true")
      .option("hoodie.index.type","GLOBAL_BLOOM")
      .mode("append")
      .save(targetHudiTablePath));

println("Saved data "+targetHudiTablePath+":"+result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

result: Unit = ()
Saved data s3://akshaya-hudi-experiments/demo/hudi/hudi_trade_info_derived_cow:()
